In [ ]:
import os
import argparse
from turtle import forward
import timm

# custom dataset
from satellitedataset import CustomDataset

import segmentation_models_pytorch as smp
from unetthreeplus import Unet3Plus
import torch
import numpy as np

from utils import label_accuracy_score, add_hist
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch.nn as nn
import torch.nn.functional as F

#import wandb
from tqdm import tqdm
import pandas as pd

import random
from torch.cuda.amp import autocast,GradScaler

In [2]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [2]:
# cuda out of memory
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

#ps aux | grep python

In [4]:
preprocessing_fn = smp.encoders.get_preprocessing_fn('efficientnet-b5','imagenet')

train_transform = A.Compose(
    [   
        A.Resize(512,512),
        A.Flip(p=0.5),
        A.ShiftScaleRotate(p=0.5),
        A.Lambda(image=preprocessing_fn),
        ToTensorV2()
    ]
)

val_transform = A.Compose([
        A.Lambda(image=preprocessing_fn),
        ToTensorV2()
])

test_transform=A.Compose([
    A.Lambda(image=preprocessing_fn),
    ToTensorV2()
])

train_dataset=CustomDataset(csv_file='./data/new_train.csv',mode='train',transform=train_transform)
train_loader=DataLoader(dataset=train_dataset,batch_size=4,shuffle=True,num_workers=4,collate_fn=collate_fn,drop_last=True)

val_dataset=CustomDataset(csv_file='./data/new_val.csv',mode='val',transform=val_transform)
val_loader=DataLoader(dataset=val_dataset,batch_size=4,shuffle=False,num_workers=4,collate_fn=collate_fn)

test_dataset=CustomDataset(csv_file='./data/test.csv',mode='test',transform=test_transform,infer=True)
test_loader=DataLoader(dataset=test_dataset,batch_size=4,shuffle=False,num_workers=4,collate_fn=collate_fn)

In [5]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
aux_params=dict(
    pooling='max',             # one of 'avg', 'max'
    dropout=0.5,               # dropout ratio, default is None
    activation = None,    # activation function, default is None
    classes=2,                 # define number of output labels
    )

model = Unet3Plus(encoder_name="resnet34" ,encoder_depth = 4,encoder_weights='imagenet',decoder_channels = 256,
        cat_channels = 64,decoder_attention_type = 'scse',in_channels=3,classes=1,aux_params=aux_params)

In [7]:
model.cuda()

Unet3Plus(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

In [9]:
import torch.nn.functional as F

#criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(params=model.parameters(), lr=5 * 1e-5, weight_decay=0.01)
print(f'Start training...')
n_class=2
best_miou=0.0
model.train()
scaler=GradScaler()

for epoch in range(10):
    optimizer.zero_grad()
    hist=np.zeros((n_class,n_class))

    for step,(images, masks) in enumerate(train_loader):
        images=torch.stack(images).float()
        masks=torch.stack(masks).long()
        
        # gpu 연산을 위해 device 할당
        images,masks=images.to(device),masks.to(device)

        with autocast():
            outputs,labels=model(images)

            #inference
            loss=0
            loss_cls = F.binary_cross_entropy_with_logits(labels,torch.amax(F.one_hot(masks.to(torch.int64),num_classes = 2).float(),dim = (1,2)))
            print(loss_cls)
            masks = F.interpolate(masks.unsqueeze(1).float(), size=outputs[0].shape[2:], mode='nearest').long().squeeze(1)

            for i in range(len(outputs)):
                print('전',outputs[i].shape)
                outputs[i] = outputs[i]*(torch.sigmoid(labels.view(-1,2,1,1))>0.5).float()
                print('후',outputs[i].shape)
                loss += F.cross_entropy(outputs[i],masks)*0.25*0.5
            loss+=loss_cls*0.4/2
            scaler.scale(loss).backward()

        if (step+1)%2==0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        
        outputs = torch.argmax(outputs[0], dim=1).detach().cpu().numpy()
        
   

    print(f'Epoch {epoch + 1}, Loss: {epoch_loss / len(dataloader)}')


Start training...


OutOfMemoryError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 15.73 GiB total capacity; 3.60 GiB already allocated; 125.81 MiB free; 3.71 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [71]:
output_path='/home/irteam/junghye-dcloud-dir/SpatialAI-Innovators/data/unet3plus.pt'
torch.save(model,output_path)

: 

In [2]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
output_path='/home/irteam/junghye-dcloud-dir/SpatialAI-Innovators/data/unet3plus.pt'
model = torch.load(output_path, map_location=device)
print(model)

Unet3Plus(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 